#### <font color="green">Output Parsers</font>
- Format the answer of the LLM.

In [ ]:
# !pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

#### <font color="green">Install LangChain</font>

In [ ]:
# pip install Langchain

#### <font color="green">Connect with an LLM</font>

In [ ]:
# !pip install Langchain-openai
# !pip install langchain-groq

###### <font color="blue">NOTE:</font>  Since right now is the best LLM in the market, We will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral Models.

##### <font color="green">LLM Model</font>

- The trend before the launch of ChatGPT-4.
- See LangChain documentation about LLM Models <a href="https://python.langchain.com/docs/integrations/chat/">here.</a>

In [5]:
from langchain_groq import ChatGroq

llmModel = ChatGroq()

##### <font color="green">Chat Model</font>
- The general trend after the launch of chatGPT-4.
  - Frequently knowns as "ChatBot".
  - Conversation between Human and AI.
  - Can have a system prompt defining the tone or the role of the AI.

- See LangChain documentation about Chat Models <a href="https://python.langchain.com/v0.1/docs/modules/model_io/chat/">here</a>

- By default we will work with ChatOpenAI. See <a href="https://python.langchain.com/v0.1/docs/integrations/chat/openai/">here</a> the LangChain documentation page about it.

In [6]:
from langchain_groq import ChatGroq

chatModel = ChatGroq(model="llama3-70b-8192")

#### <font color="green"> Parsing Outputs </font>

- See the corresponding LangChain Documentation page <a href="">here. </a>

- Language models output text. But sometimes we would like to have those answers in a different format, like a JSON dictionary or a XML document. In order to achieve that, we use the output parsers.

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser

**The previous prompt template includes the parser instructions**

In [8]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [9]:
json_chain.invoke({"question": "What is a emotion intelligence"})

{'answer': 'Emotional intelligence is the ability to understand, use, and manage your own emotions in positive ways to relieve stress, communicate effectively, empathize with others, overcome challenges and defuse conflict. It involves skills such as perceiving, using, understanding, and managing emotions.'}

**Optionally, you can use Pydantic to define a custom output format**

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

In [12]:
# Define a Pydantic object with the desired output format.
class health(BaseModel):
    setup: str = Field(description="question to set up a health")
    punchline: str = Field(description="answer to resolve the health")

In [13]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=health)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a health."})

{'setup': 'I told my wife she was drawing her eyebrows too high.',
 'punchline': 'She looked surprised.'}

In [14]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [15]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything.'}